In [1]:
import requests, zipfile, StringIO
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import pickle
from copy import deepcopy
import seaborn as sns
# Plotting Options
sns.set_style("whitegrid")
sns.despine()

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize


In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.metrics import log_loss

In [5]:
import warnings
warnings.filterwarnings("ignore") 
from IPython.core.debugger import Tracer

In [6]:
def read_csv_zip(filename):
    z = zipfile.ZipFile(filename+'.zip')
    df = pd.read_csv(z.open(filename),index_col=False)
    return df
train = read_csv_zip("train.csv")
test = read_csv_zip("test.csv")
train_sample = train.sample(frac=0.01)

In [7]:
def get_datetime(df):
    df["time_dt"] = pd.to_datetime(df["time"]*60,unit='s')
    df["year"] = df["time_dt"].apply( lambda x : x.year)
    df["month"] = df["time_dt"].apply( lambda x : x.month)
    df["day_of_week"] = df["time_dt"].apply( lambda x: x.weekday())
    df["hour"] = df["time_dt"].apply( lambda x: x.hour)
    df["day_of_year"] = df["time_dt"].apply( lambda x: x.dayofyear)

In [8]:
train_sample= train.sample(frac=0.1) #switch to full training data


In [9]:
test_sample = test.sample(frac=0.3)

In [10]:
test_data = test
msk = np.random.rand(len(train_sample)) < 0.8
train_data = train_sample[msk]
val_data = train_sample[~msk]

In [11]:
get_datetime(train_data)
get_datetime(test_data)
get_datetime(val_data)

In [12]:
train_data.head()

,row_id,x,y,accuracy,time,place_id,time_dt,year,month,day_of_week,hour,day_of_year
4637094,4637094,5.0612,3.7029,71,528316,4424198123,1971-01-02 21:16:00,1971,1,5,21,2
267449,267449,6.1940,8.6268,887,694757,3753826348,1971-04-28 11:17:00,1971,4,2,11,118
4037015,4037015,9.5382,1.8987,47,650797,6151709843,1971-03-28 22:37:00,1971,3,6,22,87
429266,429266,1.2203,2.5641,10,120233,2355236719,1970-03-25 11:53:00,1970,3,2,11,84
6154220,6154220,8.0659,4.9045,69,116720,4835025077,1970-03-23 01:20:00,1970,3,0,1,82


In [13]:
def numerical_feature_preprocess(continuous_features,train,test,val):
    scaler = StandardScaler()
    for col in continuous_features:
        #print train[col].head()
        scaler.fit(train[col])
        train[col] = scaler.transform(train[col])*fw[col]
        test[col] = scaler.transform(test[col])*fw[col]
        val[col] = scaler.transform(val[col])*fw[col]
def categorical_feature_preprocess(categorical_feature,dataset):
    list_categorical = []
    for col in categorical_feature:
        dummy = pd.get_dummies(dataset[col])
        list_categorical.append(dummy)
    dataset_categorical = pd.concat(list_categorical,axis=1)
    return dataset_categorical

In [17]:
def process_cell(train_data,val_data,test_data):
    place_encoded = LabelEncoder()
    place_encoded.fit(train_data["place_id"].append(val_data["place_id"]))
    train_data["place_encoded"] = place_encoded.transform(train_data["place_id"])
    val_data["place_encoded"] = place_encoded.transform(val_data["place_id"])

    #categorical_features = ["day_of_week"]
    numerical_feature_preprocess(continuous_features,train_data,test_data,val_data)
    #train_categorical = categorical_feature_preprocess(categorical_features,train_data)
    #test_categorical = categorical_feature_preprocess(categorical_features,test_data)
    #val_categorical = categorical_feature_preprocess(categorical_features,val_data)
    # print train_categorical.head()
    # print test_categorical.head()
    #train_data_processed = pd.concat([train_data[continuous_features],train_categorical],axis = 1)
    #test_data_processed = pd.concat([test_data[continuous_features],test_categorical],axis = 1)
    #val_data_processed = pd.concat([val_data[continuous_features],val_categorical],axis = 1)
    
    lg = LogisticRegression(random_state=20)
    lg.fit(train_data[continuous_features],train_data["place_encoded"])
    train_accuracy = np.mean(train_data["place_encoded"] == lg.predict(train_data[continuous_features]))
    val_accuracy = np.mean(val_data["place_encoded"]== lg.predict(val_data[continuous_features]))
    train_log_loss = log_loss(train_data["place_encoded"],np.array(lg.predict_proba(train_data[continuous_features])))
    #print "shape val_log_loss", val_data_processed.shape
    #print "shape val_data", val_data.shape
    #val_log_loss = log_loss(val_data["place_encoded"],np.array(lg.predict_proba(val_data_processed)))

    #print train_data_processed.shape

    test_prediction = lg.predict_proba(test_data[continuous_features])
    pred_labels = place_encoded.inverse_transform(np.argsort(test_prediction, axis=1)[:,::-1][:,:3]) 
    return pred_labels,train_accuracy,train_log_loss,val_accuracy #test_prediction,pred_labels
    
    
    

In [18]:

def process_grid(train_data,val_data,test_data,x_step,y_step,xy_range):
    preds = np.zeros((test_data.shape[0], 3), dtype=np.int64)
    #preds_val = np.zeros((val_data.shape[0],3),dtype=np.int64)
    #print test_data.shape[0]
    train_accuracy = np.zeros((len(xy_range),len(xy_range)),dtype=float)
    val_accuracy = np.zeros((len(xy_range),len(xy_range)),dtype=float)
    train_log_loss = np.zeros((len(xy_range),len(xy_range)),dtype=float)
    val_log_loss = np.zeros((len(xy_range),len(xy_range)),dtype=float)
    i, j = 0, 0
    for x in xy_range:
        j = 0
        for y in xy_range:
            train_data_cell = train_data.loc[(train_data["x"] >= (x-x_step)) & (train_data["x"] <= (x)) & (train_data["y"] >= (y-y_step)) & (train_data["y"] <= (y))] 
            # print train_data_cell.head()
            test_data_cell = test_data.loc[(test_data["x"] >= (x-x_step)) & (test_data["x"] <= (x)) & (test_data["y"] >= (y-y_step)) & (test_data["y"] <= (y))] 
            val_data_cell = val_data.loc[(val_data["x"] >= (x-x_step)) & (val_data["x"] <= (x)) & (val_data["y"] >= (y-y_step)) & (val_data["y"] <= (y))]
            
            row_ids = test_data_cell.index
            row_ids_val = val_data_cell.index
            pred_labels,train_accuracy[i,j],train_log_loss[i,j],val_accuracy[i,j] = process_cell(train_data_cell,val_data_cell,test_data_cell)
            #print max(row_ids)
            #print test_data.shape
            preds[row_ids] = pred_labels
            # print x,y
            # print i,j
            j += 1
        i += 1
    pre_result = pd.DataFrame(preds,dtype = str,columns=["l1","l2","l3"])
    result = pre_result.l1.str.cat([pre_result.l2,pre_result.l3],sep = " ")
    result.name = "place_id"
    #result.to_csv("fb_checkin.csv",index=True, header=True, index_label = "row_id")
    #print result.head()
    #print train_accuracy
    #print train_log_loss
    return result,train_accuracy,train_log_loss,val_accuracy
    
    

In [19]:
file_path = "/Users/ChiYuan/Documents/python/Kaggle/Facebook Checkin/"
continuous_features = ["x","y","time","year","month","hour","day_of_year","day_of_week","accuracy"]
#weight for ["x","y","time","year","month","hour","day_of_year","day_of_week","accuracy"]
fw = {"x":1,"y":2,"time":0.5,"year":0.5,"month":0.5,"hour":0.5,"day_of_year":0.2,"day_of_week":0.5,"accuracy":0.1} #wait is apply after each feature is transformed by standard deviation
x_step = 0.5
y_step = 0.5
xy_range = np.linspace(0.5, 10.0, 20)
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
result,train_accuracy,train_log_loss,val_accuracy = process_grid(train_data,val_data,test_data,x_step,y_step,xy_range)


In [20]:
val_accuracy

array([[ 0.17780503,  0.16553288,  0.16619519,  0.17525126,  0.16015375,
         0.196793  ,  0.1661195 ,  0.18187919,  0.17280453,  0.1719697 ,
         0.16582187,  0.2115942 ,  0.18012924,  0.19554849,  0.18838174,
         0.1813054 ,  0.16240725,  0.14037433,  0.19373777,  0.183687  ],
       [ 0.16081871,  0.18044515,  0.16918647,  0.16145181,  0.15826331,
         0.16242822,  0.18309859,  0.1595359 ,  0.16131026,  0.160401  ,
         0.15251799,  0.15424165,  0.18215339,  0.18780096,  0.17937545,
         0.15427658,  0.1663067 ,  0.16049383,  0.17156528,  0.16741214],
       [ 0.17873754,  0.19859813,  0.15147625,  0.14183381,  0.15529248,
         0.15268558,  0.16970138,  0.18539326,  0.14371257,  0.13560454,
         0.17044674,  0.17777778,  0.16065574,  0.12953368,  0.16690751,
         0.16838906,  0.17747164,  0.16224189,  0.15946844,  0.15801024],
       [ 0.19754867,  0.1673699 ,  0.15825688,  0.17238913,  0.15250696,
         0.157277  ,  0.15896359,  0.1663286 ,  

In [21]:
train_accuracy

array([[ 0.20233645,  0.19209576,  0.2012963 ,  0.19235794,  0.18149524,
         0.21189265,  0.18988983,  0.20108505,  0.19438559,  0.1978022 ,
         0.19728507,  0.21304586,  0.19429134,  0.21615889,  0.2153059 ,
         0.20916092,  0.18222482,  0.17265434,  0.21843854,  0.19837047],
       [ 0.18562232,  0.21015348,  0.19006479,  0.1863306 ,  0.1747473 ,
         0.19280311,  0.18007025,  0.19641907,  0.17710168,  0.20022753,
         0.18342887,  0.19504542,  0.19250577,  0.18526877,  0.18561312,
         0.19781962,  0.18365142,  0.18318318,  0.19071251,  0.18442895],
       [ 0.20234899,  0.20235383,  0.17231076,  0.16571952,  0.19092146,
         0.18054692,  0.18172502,  0.20201159,  0.18229694,  0.16339869,
         0.18188252,  0.20301304,  0.17823156,  0.15629602,  0.19021837,
         0.191833  ,  0.19411421,  0.19800269,  0.15907591,  0.19565621],
       [ 0.20326332,  0.19158323,  0.18397018,  0.19279846,  0.17282025,
         0.18736802,  0.19705992,  0.16423796,  

In [ ]:
result.to_csv(file_path+"fb_checkin.csv",index=True, header=True, index_label = "row_id")
    #print result.head()

In [ ]:
print xy_range

In [ ]:
train_data.head(10)